# 📓 Groundedness Evaluations

In many ways, feedbacks can be thought of as LLM apps themselves. Given text, they return some result. Thinking in this way, we can use TruLens to evaluate and track our feedback quality. We can even do this for different models or prompting schemes (such as chain-of-thought reasoning). In particular, here we will evaluate the latest open source LLM published by Snowflake, [`arctic-instruct`](https://huggingface.co/Snowflake/snowflake-arctic-instruct), a 480B dense-MOE hybrid transformer-based model.

This notebook follows an evaluation of a set of test cases generated from human annotated datasets. In particular, we generate test cases from [SummEval](https://arxiv.org/abs/2007.12626).

SummEval is one of the datasets dedicated to automated evaluations on summarization tasks, which are closely related to the groundedness evaluation in RAG with the retrieved context (i.e. the source) and response (i.e. the summary). It contains human annotation of numerical score (**1** to **5**) comprised of scoring from 3 human expert annotators and 5 crowd-sourced annotators. There are 16 models being used for generation in total for 100 paragraphs in the test set, so there are a total of 16,000 machine-generated summaries. Each paragraph also has several human-written summaries for comparative analysis. 


For evaluating groundedness feedback functions, we compute the annotated "consistency" scores, a measure of whether the summarized response is factually consistent with the source texts and hence can be used as a proxy to evaluate groundedness in our RAG triad, and normalized to **0** to **1** score as our **expected_score** and to match the output of feedback functions.

In [ ]:
# Import groundedness feedback function
from test_cases import generate_summeval_groundedness_golden_set
from trulens.apps.basic import TruBasicApp
from trulens.core import Feedback
from trulens.core import Select
from trulens.core import TruSession
from trulens.feedback import GroundTruthAgreement

tru = TruSession()
tru.reset_database()

# generator for groundedness golden set
test_cases_gen = generate_summeval_groundedness_golden_set(
    "./datasets/summeval/summeval_test_100.json"
)

In [ ]:
# specify the number of test cases we want to run the smoke test on
groundedness_golden_set = []
for i in range(500):
    groundedness_golden_set.append(next(test_cases_gen))

In [ ]:
len(groundedness_golden_set)

In [ ]:
groundedness_golden_set[:5]

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."
os.environ["REPLICATE_API_TOKEN"] = "r8_..."

### Benchmarking various Groundedness feedback function providers (OpenAI GPT-4o vs Snowflake's Arctic Instruct)

In [ ]:
from trulens.providers.litellm import LiteLLM
from trulens.providers.openai import OpenAI

replicate_provider_arctic = LiteLLM(
    model_engine="replicate/snowflake/snowflake-arctic-instruct"
)

f_groundedness_arctic = Feedback(
    replicate_provider_arctic.groundedness_measure_with_cot_reasons,
    name="Groundedness arctic-instruct",
).on_input_output()


def wrapped_groundedness_arctic(input, output) -> float:
    score = f_groundedness_arctic(input, output)[0]
    print(score)
    return score


replicate_provider_llama3 = LiteLLM(
    model_engine="replicate/meta/meta-llama-3-70b-instruct"
)

f_groundedness_llama3 = Feedback(
    replicate_provider_llama3.groundedness_measure_with_cot_reasons,
    name="Groundedness Llama-3-70b-instruct",
).on_input_output()


def wrapped_groundedness_llama3(input, output) -> float:
    score = f_groundedness_llama3(input, output)[0]
    print(score)
    return score


replicate_provider_mixtral = LiteLLM(
    model_engine="replicate/mistralai/mixtral-8x7b-instruct-v0.1"
)
f_groundedness_mixtral = Feedback(
    replicate_provider_mixtral.groundedness_measure_with_cot_reasons,
    name="Groundedness Mixtral-8x7b-instruct-v0.1",
).on_input_output()


def wrapped_groundedness_mixtral(input, output) -> float:
    score = f_groundedness_mixtral(input, output)[0]
    print(score)
    return score


openai_provider_turbo = OpenAI(model_engine="gpt-4-turbo")
f_groundedness_openai_gpt4_turbo = Feedback(
    openai_provider_turbo.groundedness_measure_with_cot_reasons,
    name="Groundedness OpenAI GPT-4-turbo",
).on_input_output()


def wrapped_groundedness_openai_gpt4_turbo(input, output) -> float:
    return f_groundedness_openai_gpt4_turbo(input, output)[0]


openai_provider = OpenAI(model_engine="gpt-4o")
f_groundedness_openai_gpt4 = Feedback(
    openai_provider.groundedness_measure_with_cot_reasons,
    name="Groundedness OpenAI GPT-4o",
).on_input_output()


def wrapped_groundedness_openai_gpt4(input, output) -> float:
    return f_groundedness_openai_gpt4(input, output)[0]


# Create a Feedback object using the numeric_difference method of the ground_truth object
ground_truth = GroundTruthAgreement(
    groundedness_golden_set, provider=openai_provider
)
# Call the numeric_difference method with app and record and aggregate to get the mean absolute error
f_absolute_error = (
    Feedback(ground_truth.absolute_error, name="Mean Absolute Error")
    .on(Select.Record.calls[0].args.args[0])
    .on(Select.Record.calls[0].args.args[1])
    .on_output()
)

In [ ]:
tru_wrapped_groundedness_llama3 = TruBasicApp(
    wrapped_groundedness_llama3,
    app_name="groundedness",
    app_version="Llama3-70b-instruct",
    feedbacks=[f_absolute_error],
)
for i in range(len(groundedness_golden_set)):
    source = groundedness_golden_set[i]["query"]
    response = groundedness_golden_set[i]["response"]

    with tru_wrapped_groundedness_llama3 as recording:
        try:
            tru_wrapped_groundedness_llama3.app(source, response)
        except Exception as e:
            print(e)

In [ ]:
tru_wrapped_groundedness_mixtral = TruBasicApp(
    wrapped_groundedness_mixtral,
    app_name="groundedness",
    app_version="mixtral-8x7b-instruct-v0.1",
    feedbacks=[f_absolute_error],
)
for i in range(len(groundedness_golden_set)):
    source = groundedness_golden_set[i]["query"]
    response = groundedness_golden_set[i]["response"]

    with tru_wrapped_groundedness_mixtral as recording:
        try:
            tru_wrapped_groundedness_mixtral.app(source, response)
        except Exception as e:
            print(e)

In [ ]:
tru.get_leaderboard()